# Multi-outputs Task

In [ ]:
!pip install polars[pandas]

In [1]:
import numpy
import pandas # or use `polars`
import torch

## Data Processing from scikit-learn

In [2]:
from sklearn.datasets import make_multilabel_classification

In [3]:
X, y = make_multilabel_classification(n_samples=1000, 
                                      n_features=10,
                                      n_classes=3,
                                      n_labels=2,
                                      random_state=1)

In [4]:
X.shape, y.shape

((1000, 10), (1000, 3))

## Machine Learning Process

### Load Perming and Config Hyperparameters


In [5]:
import perming
main = perming.Box(10, 3, (30,), batch_size=8, activation='relu', inplace_on=True, solver='sgd', criterion="MultiLabelSoftMarginLoss", learning_rate_init=0.01)
# main = perming.Ranker(10, 3, (30,), batch_size=16, activation='relu', solver='sgd', criterion="MultiLabelSoftMarginLoss", learning_rate_init=0.01)
# main = perming.COMMON_MODELS['Multi-outputs'](10, 3, (30,), batch_size=16, activation='relu', solver='sgd', criterion="MultiLabelSoftMarginLoss", learning_rate_init=0.01)
main.print_config()

MLP(
  (mlp): Sequential(
    (Linear0): Linear(in_features=10, out_features=30, bias=True)
    (Activation0): ReLU(inplace=True)
    (Linear1): Linear(in_features=30, out_features=3, bias=True)
  )
)


OrderedDict([('torch -v', '1.7.1+cu101'),
             ('criterion', MultiLabelSoftMarginLoss()),
             ('batch_size', 8),
             ('solver',
              SGD (
              Parameter Group 0
                  dampening: 0
                  lr: 0.01
                  momentum: 0
                  nesterov: False
                  weight_decay: 0
              )),
             ('lr_scheduler', None),
             ('device', device(type='cuda'))])

### Dataloader from Numpy with Multi-threaded


In [6]:
main.data_loader(X, y, random_seed=0)

### Training Stage and Accelerated Validation


In [7]:
main.train_val(num_epochs=40, interval=10, tolerance=1e-4, patience=10, early_stop=True)

Epoch [1/40], Step [10/100], Training Loss: 0.8111, Validation Loss: 0.6473
Epoch [1/40], Step [20/100], Training Loss: 0.6409, Validation Loss: 0.6319
Epoch [1/40], Step [30/100], Training Loss: 0.5722, Validation Loss: 0.6216
Epoch [1/40], Step [40/100], Training Loss: 0.5732, Validation Loss: 0.6015
Epoch [1/40], Step [50/100], Training Loss: 0.7571, Validation Loss: 0.5837
Epoch [1/40], Step [60/100], Training Loss: 0.5953, Validation Loss: 0.5720
Epoch [1/40], Step [70/100], Training Loss: 0.4726, Validation Loss: 0.5622
Epoch [1/40], Step [80/100], Training Loss: 0.5246, Validation Loss: 0.5515
Epoch [1/40], Step [90/100], Training Loss: 0.6551, Validation Loss: 0.5410
Epoch [1/40], Step [100/100], Training Loss: 0.4357, Validation Loss: 0.5401
Epoch [2/40], Step [10/100], Training Loss: 0.6371, Validation Loss: 0.5278
Epoch [2/40], Step [20/100], Training Loss: 0.4437, Validation Loss: 0.5240
Epoch [2/40], Step [30/100], Training Loss: 0.5097, Validation Loss: 0.5322
Epoch [2/40

### Test Model with Loss


In [8]:
main.test()

loss of Box on the 104 test dataset: 0.18220241367816925.


OrderedDict([('problem', 'multi-outputs'),
             ('loss',
              {'train': 0.32005155086517334,
               'val': 0.26151183247566223,
               'test': 0.18220241367816925})])

In [9]:
X, y = torch.as_tensor(X, dtype=torch.float).to(torch.device("cuda")), torch.as_tensor(y, dtype=torch.float).to(torch.device("cuda"))

In [10]:
pred = main.model(X) # predicted
# refer to https://pytorch.org/torcheval/main/ for metrics functional tools, like classification
# take input as pred, target as y

In [11]:
'{}%'.format(100 * sum(row.all().int().item() for row in (pred.ge(0.5) == y)) / X.shape[0])

'75.9%'

### Save Model Parameters to Models Folder


In [12]:
main.save(show=False, dir='../models/outputs.ckpt')

### Load Model Parameters from Models Folder

In [13]:
main.load(show=False, dir='../models/outputs.ckpt')